https://huggingface.co/blog/how-to-train

In [7]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path("./all-datasets/").glob("**/*.txt")]

paths

['all-datasets/kk-1.txt',
 'all-datasets/kk_part_2.txt',
 'all-datasets/kk.txt',
 'all-datasets/kaz_wikipedia_2016_300K-sentences.txt',
 'all-datasets/kaz_newscrawl_2016_1M-sentences.txt']

In [8]:
# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save files to disk
tokenizer.save_model(".", "kaz-tokenizer")

['./kaz-tokenizer-vocab.json', './kaz-tokenizer-merges.txt']

In [13]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./kaz-tokenizer-vocab.json",
    "./kaz-tokenizer-merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

encoded = tokenizer.encode("1000 метрге төрт адамдық байдарка қайықпен жарысуда Қазақстан құрамасы алтын медаль алды.")
print(
    encoded
)

Encoding(num_tokens=19, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [14]:
encoded.tokens

['<s>',
 '1000',
 'ĠÐ¼ÐµÑĤÑĢÐ³Ðµ',
 'ĠÑĤÓ©ÑĢÑĤ',
 'ĠÐ°Ð´Ð°Ð¼Ð´ÑĭÒĽ',
 'ĠÐ±Ð°Ð¹',
 'Ð´Ð°ÑĢ',
 'ÐºÐ°',
 'ĠÒĽÐ°Ð¹ÑĭÒĽ',
 'Ð¿ÐµÐ½',
 'ĠÐ¶Ð°ÑĢÑĭÑģ',
 'ÑĥÐ´Ð°',
 'ĠÒļÐ°Ð·Ð°ÒĽÑģÑĤÐ°Ð½',
 'ĠÒĽÒ±ÑĢÐ°Ð¼Ð°ÑģÑĭ',
 'ĠÐ°Ð»ÑĤÑĭÐ½',
 'ĠÐ¼ÐµÐ´Ð°Ð»ÑĮ',
 'ĠÐ°Ð»Ð´Ñĭ',
 '.',
 '</s>']

In [15]:
from tokenizers.decoders import ByteLevel
decoder = ByteLevel()

kaz_tokens = []
for t in encoded.tokens:
    kaz_tokens.append(decoder.decode(t))
print(kaz_tokens)

['<s>', '1000', ' метрге', ' төрт', ' адамдық', ' бай', 'дар', 'ка', ' қайық', 'пен', ' жарыс', 'уда', ' Қазақстан', ' құрамасы', ' алтын', ' медаль', ' алды', '.', '</s>']
